In [70]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

In [71]:
loc = "./Premier-League/"

teams = pd.read_csv(loc + "EPLStandindsFinal.csv")["Team"]
print(teams[25])

dataset = pd.read_csv(loc + "premier_league2001-2021.csv")
dataset["Date"] = pd.to_datetime(dataset["Date"])
dataset.info()


Man City
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7980 entries, 0 to 7979
Data columns (total 51 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           7980 non-null   datetime64[ns]
 1   HomeTeam       7980 non-null   object        
 2   AwayTeam       7980 non-null   object        
 3   FTHG           7980 non-null   int64         
 4   FTAG           7980 non-null   int64         
 5   FTR            7980 non-null   object        
 6   HTGS           7980 non-null   int64         
 7   ATGS           7980 non-null   int64         
 8   HTGC           7980 non-null   int64         
 9   ATGC           7980 non-null   int64         
 10  HTP            7980 non-null   float64       
 11  ATP            7980 non-null   float64       
 12  HM1            7980 non-null   object        
 13  HM2            7980 non-null   object        
 14  HM3            7980 non-null   object        
 15  HM4         

In [72]:
dataset["target"] = (dataset["FTR"] == "H").astype('int')
dataset

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffFormPts3,DiffLP,target
0,2000-08-19,Charlton,Man City,4,0,H,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0,0,0.0,1
1,2000-08-19,Chelsea,West Ham,4,2,H,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0,0,-4.0,1
2,2000-08-19,Coventry,Middlesbrough,1,3,A,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0,0,2.0,0
3,2000-08-19,Derby,Southampton,2,2,D,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0,0,1.0,0
4,2000-08-19,Leeds,Everton,2,0,H,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0,0,-10.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,2021-05-23,Liverpool,Crystal Palace,2,0,H,66,41,42,64,...,0,0,0,0.631579,-0.605263,0.578947,7,6,-13.0,1
7976,2021-05-23,Man City,Everton,5,0,H,78,47,32,43,...,0,0,0,1.210526,0.105263,0.631579,2,-1,-10.0,1
7977,2021-05-23,Sheffield United,Burnley,1,0,H,19,33,63,54,...,0,0,0,-1.157895,-0.552632,-0.500000,0,0,-1.0,1
7978,2021-05-23,West Ham,Southampton,3,0,H,59,47,47,65,...,0,0,0,0.315789,-0.473684,0.500000,0,-2,5.0,1


In [73]:
dataset["HomeTeamCode"] = [teams.index[teams == team][0] for team in dataset["HomeTeam"]]
dataset

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffFormPts3,DiffLP,target,HomeTeamCode
0,2000-08-19,Charlton,Man City,4,0,H,0,0,0,0,...,0,0,0.000000,0.000000,0.000000,0,0,0.0,1,12
1,2000-08-19,Chelsea,West Ham,4,2,H,0,0,0,0,...,0,0,0.000000,0.000000,0.000000,0,0,-4.0,1,13
2,2000-08-19,Coventry,Middlesbrough,1,3,A,0,0,0,0,...,0,0,0.000000,0.000000,0.000000,0,0,2.0,0,14
3,2000-08-19,Derby,Southampton,2,2,D,0,0,0,0,...,0,0,0.000000,0.000000,0.000000,0,0,1.0,0,16
4,2000-08-19,Leeds,Everton,2,0,H,0,0,0,0,...,0,0,0.000000,0.000000,0.000000,0,0,-10.0,1,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,2021-05-23,Liverpool,Crystal Palace,2,0,H,66,41,42,64,...,0,0,0.631579,-0.605263,0.578947,7,6,-13.0,1,24
7976,2021-05-23,Man City,Everton,5,0,H,78,47,32,43,...,0,0,1.210526,0.105263,0.631579,2,-1,-10.0,1,25
7977,2021-05-23,Sheffield United,Burnley,1,0,H,19,33,63,54,...,0,0,-1.157895,-0.552632,-0.500000,0,0,-1.0,1,33
7978,2021-05-23,West Ham,Southampton,3,0,H,59,47,47,65,...,0,0,0.315789,-0.473684,0.500000,0,-2,5.0,1,41


In [74]:
dataset["AwayTeamCode"] = [teams.index[teams == team][0] for team in dataset["AwayTeam"]]
dataset

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffFormPts3,DiffLP,target,HomeTeamCode,AwayTeamCode
0,2000-08-19,Charlton,Man City,4,0,H,0,0,0,0,...,0,0.000000,0.000000,0.000000,0,0,0.0,1,12,25
1,2000-08-19,Chelsea,West Ham,4,2,H,0,0,0,0,...,0,0.000000,0.000000,0.000000,0,0,-4.0,1,13,41
2,2000-08-19,Coventry,Middlesbrough,1,3,A,0,0,0,0,...,0,0.000000,0.000000,0.000000,0,0,2.0,0,14,27
3,2000-08-19,Derby,Southampton,2,2,D,0,0,0,0,...,0,0.000000,0.000000,0.000000,0,0,1.0,0,16,34
4,2000-08-19,Leeds,Everton,2,0,H,0,0,0,0,...,0,0.000000,0.000000,0.000000,0,0,-10.0,1,22,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,2021-05-23,Liverpool,Crystal Palace,2,0,H,66,41,42,64,...,0,0.631579,-0.605263,0.578947,7,6,-13.0,1,24,15
7976,2021-05-23,Man City,Everton,5,0,H,78,47,32,43,...,0,1.210526,0.105263,0.631579,2,-1,-10.0,1,25,17
7977,2021-05-23,Sheffield United,Burnley,1,0,H,19,33,63,54,...,0,-1.157895,-0.552632,-0.500000,0,0,-1.0,1,33,10
7978,2021-05-23,West Ham,Southampton,3,0,H,59,47,47,65,...,0,0.315789,-0.473684,0.500000,0,-2,5.0,1,41,34


In [75]:
logistical_regression = LogisticRegression()

x = dataset[["HomeTeamCode", "AwayTeamCode", "DiffFormPts", "DiffLP"]]
y = dataset["target"]

x_train = x[:6000]
y_train = y[:6000]
x_test = x[6000:]
y_test = y[6000:]

logistical_regression.fit(x_train, y_train)

yhat = logistical_regression.predict(x_test)
print("Acurácia: ", accuracy_score(y_test, yhat))
print("Precisão: ", precision_score(y_test, yhat))

Acurácia:  0.6515151515151515
Precisão:  0.6240963855421687


In [76]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date")
    rolling_stats = group[cols].rolling(3, closed='left').sum()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [77]:
cols = ["FTHG", "FTAG"]
newcols = ["L3MGS", "L3MGC"]
dataset_rolling = dataset.groupby("HomeTeam").apply(lambda x: rolling_averages(x, cols, newcols))
dataset_rolling

Date HomeTeam                AwayTeam  FTHG  FTAG FTR  \
HomeTeam                                                                    
Arsenal  76   2000-10-01  Arsenal              Man United     1     0   H   
         79   2000-10-14  Arsenal             Aston Villa     1     0   H   
         99   2000-10-28  Arsenal                Man City     5     0   H   
         119  2000-11-11  Arsenal                   Derby     0     0   D   
         149  2000-12-02  Arsenal             Southampton     1     0   H   
...                  ...      ...                     ...   ...   ...  ..   
Wolves   7899 2021-04-05   Wolves                West Ham     2     3   A   
         7912 2021-04-17   Wolves        Sheffield United     1     0   H   
         7924 2021-04-25   Wolves                 Burnley     0     4   A   
         7942 2021-05-09   Wolves  Brighton & Hove Albion     2     1   H   
         7979 2021-05-23   Wolves              Man United     1     2   A   

               HTGS  ATGS  HTGC  ATGC  ...      ATGD   DiffPts DiffFormPts  \
HomeTeam                               ...                                   
Arsenal  76      13    20     9     7  ...  1.625000 -0.375000          -2   
         79      14    11     9     7  ...  0.500000  0.000000          -1   
         99      17    14    10    14  ...  0.000000  0.700000           6   
         119     23    16    10    28  ... -1.000000  1.583333          12   
         149     23    21    13    27  ... -0.400000  0.733333           3   
...             ...   ...   ...   ...  ...       ...       ...         ...   
Wolves   7899    28    45    38    35  ...  0.333333 -0.466667          -2   
         7912    31    17    41    55  ... -1.187500  0.750000           1   
         7924    32    26    41    45  ... -0.575758  0.242424           3   
         7942    33    35    46    39  ... -0.114286  0.142857           2   
         7979    35    71    50    43  ...  0.736842 -0.684211          -1   

              DiffFormPts3 DiffLP target HomeTeamCode AwayTeamCode L3MGS L3MGC  
HomeTeam                                                                        
Arsenal  76             -2    1.0      1            0           26   9.0   4.0  
         79             -2   -4.0      1            0            1   8.0   4.0  
         99              3  -16.0      1            0           25   4.0   1.0  
         119             9  -14.0      0            0           16   7.0   0.0  
         149            -3  -13.0      1            0           34   6.0   0.0  
...                    ...    ...    ...          ...          ...   ...   ...  
Wolves   7899           -3   -9.0      0           43           41   1.0   1.0  
         7912            3   -2.0      1           43           33   3.0   4.0  
         7924            6   -3.0      0           43           10   3.0   4.0  
         7942            0   -8.0      1           43            9   3.0   7.0  
         7979            2    4.0      0           43           26   3.0   5.0  

[7851 rows x 56 columns]

In [78]:
x = dataset_rolling[["HomeTeamCode", 
                     "AwayTeamCode", 
                     "DiffFormPts3", 
                     "DiffLP", 
                     "L3MGS",
                     "L3MGC"]]

y = dataset_rolling["target"]

x_train = x[dataset_rolling["Date"] < '2016-01-01']
y_train = y[dataset_rolling["Date"] < '2016-01-01']

x_test = x[dataset_rolling["Date"] > '2016-01-01']
y_test = y[dataset_rolling["Date"] > '2016-01-01']

logistical_regression = LogisticRegression()

logistical_regression.fit(x_train, y_train)

yhat = logistical_regression.predict(x_test)

print("Acurácia: ", accuracy_score(y_test, yhat))
print("Precisão: ", precision_score(y_test, yhat))

Acurácia:  0.6453934740882917
Precisão:  0.6137142857142858


In [79]:
x = dataset_rolling[["HomeTeamCode", 
                     "AwayTeamCode", 
                     "DiffFormPts", 
                     "DiffFormPts3",
                     "HTWinStreak3",
                     "HTWinStreak5",
                     "HTLossStreak3",
                     "HTLossStreak5",
                     "ATWinStreak3",
                     "ATWinStreak5",
                     "ATLossStreak3",
                     "ATLossStreak5",
                     "DiffLP", 
                     "L3MGS",
                     "L3MGC"]]

y = dataset_rolling["target"]

x_train = x[dataset_rolling["Date"] < '2016-01-01']
y_train = y[dataset_rolling["Date"] < '2016-01-01']

x_test = x[dataset_rolling["Date"] > '2016-01-01']
y_test = y[dataset_rolling["Date"] > '2016-01-01']

logistical_regression = LogisticRegression()

logistical_regression.fit(x_train, y_train)

yhat = logistical_regression.predict(x_test)

print("Acurácia: ", accuracy_score(y_test, yhat))
print("Precisão: ", precision_score(y_test, yhat))

Acurácia:  0.6573896353166987
Precisão:  0.6258426966292134


/home/walrusbane/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
